### 영화 정보 crawling

In [1]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import csv
import re
import pandas as pd
import numpy as np

# page_dic = {'1990': 17, '1991' : 16, '1992' : 20, '1993': 21, '1994' : 19, '1995' : 17, '1996' : 17, '1997' : 17, '1998' : 15, '1999' : 16, 
#             '2000' : 19, '2001' : 17, '2002' : 18, '2003' : 20, '2004' : 38, '2005' : 21, '2006' : 24, '2007' : 26, '2008' : 28, '2009' : 24,
#             '2010' : 29, '2011' : 41, '2012' : 44, '2013' : 61, '2014' : 76, '2015' : 71, '2016' : 59, '2017' : 53, '2018' : 50, '2019' : 43,
#             '2020' : 37, '2021': 38}

# page_dic = {'2014':76,'2015':71}
page_dic = {'2014':76,'2015':71,
            '2016' : 59, '2017' : 53, '2018' : 50, '2019' : 43,'2020' : 37,}
def load_soup(dic):
    years = list(dic.keys())
    pages = list(dic.values())

    soup_object = []
    for idx, year in enumerate(years): 
        for page in range(1, pages[idx]+1):
            # print(year ,page)
            base_url = f"https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open={year}&page={str(page)}"
            response = requests.get(base_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            data_select = soup.select("#old_content > ul > li")
            soup_object.append(data_select)
    return soup_object


def load_movie_id(dic):
    movie_id = []

    for movie_select in load_soup(dic):
        # print(movie_select)
        for data in movie_select:
            movie_id.append(data.select_one('a')['href'].split('=')[1])
    return movie_id


def load_url(subject, movie_num):
    base_url = f'https://movie.naver.com/movie/bi/mi/{subject}.nhn?code={movie_num}'
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    article = soup.select_one('#content > div.article')
    return article

# page_dic2 = {'1990': 17, '1991' : 16}

movie_id = load_movie_id(page_dic)
# print(movie_id)
print(len(movie_id))

for idx, movie_num in enumerate(movie_id):
    # print(idx+1)
    basic_article = load_url('basic', movie_num) # 줄거리를 보여주는 페이지
    datail_article = load_url('detail', movie_num) # 출연배우를 보여주는 페이지

    # 19 청불영화에 대한 예외처리
    if basic_article is None:
        continue
    
    # 영화제목
    title = basic_article.select_one('div.mv_info_area > div.mv_info > h3').get_text().replace('\n', '')
    # print(title)
    posters = basic_article.select('div.mv_info_area > div.poster> a>img ')
    image_link=[]
    for poster in posters:
        image  = poster['src']
        image_link.append(image)
    # 네티즌 점수 
    star_lst = []
    for num in range(1,5):
        if basic_article.select_one(f'div.mv_info_area > div.mv_info > div.main_score > div.score.score_left > div.star_score > a > em:nth-of-type({num})') is not None:
            star = basic_article.select_one(f'div.mv_info_area > div.mv_info > div.main_score > div.score.score_left > div.star_score > a > em:nth-of-type({num})').get_text()
            star_lst.append(str(star))
    # print("".join(i for i in star_lst))

    # 영화 정보리스트
    info_lst = [i.get_text() for i in basic_article.select('div.mv_info_area > div.mv_info > dl > dt')]
    # print(info_lst)

    # 영화 장르,년도
    if '개요()' in info_lst:
        idx = info_lst.index('개요()')
        genre = [i.get_text() for i in basic_article.select(f'div.mv_info_area > div.mv_info > dl > dd:nth-of-type({idx+1}) > p > span:nth-of-type(1) > a')]
        years = [i.get_text() for i in basic_article.select(f'div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(4) > a:nth-child(1)')]
    else:
        genre = ""
        years = ""
    # print(genre)

    # 영화 감독
    if '감독' in info_lst:
        idx = info_lst.index('감독')
        if basic_article.select_one(f'div.mv_info_area > div.mv_info > dl > dd:nth-of-type({idx+1}) > p > a') is not None:
            movie_director = basic_article.select_one(f'div.mv_info_area > div.mv_info > dl > dd:nth-of-type({idx+1}) > p > a').get_text()
    else:
        movie_director = ""
    # print(movie_director)
    
    # 영화 등급
    if '등급' in info_lst:
        idx = info_lst.index('등급')
        movie_rating = basic_article.select_one(f'div.mv_info_area > div.mv_info > dl > dd:nth-of-type({idx+1}) > p > a').get_text()
    else:
        movie_rating = ""
    # print(movie_rating)

    # 영화 출연
    movie_actors = datail_article.select('div.section_group.section_group_frst > div.obj_section.noline > div > div.lst_people_area.height100')
    #content > div.article > div.section_group.section_group_frst > div.obj_section.noline > div > div.lst_people_area.height100
    actor_lst = []
    # 영화배우 count
    actor_counts = datail_article.select('div.section_group.section_group_frst > div.obj_section.noline > div > div.lst_people_area.height100 > ul > li')
    actor_counts = len(actor_counts)

    for actors in movie_actors:
        for num in range(1, actor_counts+1):
            if actors.select_one(f'ul > li:nth-of-type({num}) > div > div > p.in_prt > em').get_text() == '주연':
                if actors.select_one(f'ul > li:nth-of-type({num}) > div > a') is not None:
                    actor = actors.select_one(f'ul > li:nth-of-type({num}) > div > a').get_text()
                    actor_lst.append(actor)
                elif actors.select_one(f'ul > li:nth-of-type({num}) > div > span') is not None:
                    actor = actors.select_one(f'ul > li:nth-of-type({num}) > div > span').get_text()
                    actor_lst.append(actor)
            else:
                break
            
    # 영화 줄거리
    if basic_article.select_one('div.section_group.section_group_frst > div:nth-of-type(1) > div > div > div > h4') is None:
        summary = ""
    else:
        summary = basic_article.select_one('div.section_group.section_group_frst > div:nth-of-type(1) > div > div.story_area > p').get_text().replace('\r', "").replace('\xa0', "")
    # print(summary)
    
    movie_data = {
        'movie_id' : movie_num,
        'title' : title,
        'years': "".join(year for year in years),
        'star' : "".join(i for i in star_lst),
        'movie_rating' : movie_rating,
        'genre' : "".join(genres for genres in genre),
        'director' : movie_director,
        'actors' : "".join(actor for actor in actor_lst),
        'summary' : summary,
        'image_link' :"".join(i for i in image_link)
    }

    #print(movie_data, '\n')

    with open('./movie_data5.csv', 'a', encoding='utf-8', newline='') as csvfile:
        fieldnames = ['movie_id', 'title', 'years', 'star', 'movie_rating',  'genre', 'director', 'actors', 'summary','image_link']
        csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        csvwriter.writerow(df)

7725


In [24]:
df.head()

,19436,007 북경특급 2,2014,7.80,15세 관람가,"['코미디', '액션']",주성치,['주성치'],"옛날 중국에선 황제의 보위를 위해 비밀 훈련을 받은 네 인물을 뽑아 보룡일족이라고 불렀다. 보룡일족 네 사람인 공, 희, 발, 제 네사람 중 공, 희, 제는 무술에 관한한 나라안의 최고 귀재였지만 발만은 무술에는 문외한이다. 원래 지방의 의사였던 발은 머리를 짜내 황제에게 바칠 발명품들을 만들어냈다. 암행어사 마패모양의 자석 방어기구와 황제를 위한 초대형 선풍기가 그것. 그러나 황상은 그를 중히 여기지 않고, 다만 발에게 신분을 숨기고 민심을 살피는 임무를 주는데……",https://movie-phinf.pstatic.net/20210405_182/16176099597336iQsr_JPEG/movie_image.jpg?type=m203_290_2
0,129765,1000 : 최후의 전사들,2014.0,6.63,15세 관람가,"['액션', '모험', '드라마']",아칸 사타예브,"['아실칸 톨예포프', '아얀 우테부르겐']",칭기즈칸 제국을 재건하려는 준가 족은 카자흐 민족을 학살하고 폭정을 일삼는다. 가...,https://movie-phinf.pstatic.net/20141014_164/1...
1,120699,100인의 뷰티온,NaN,0.00,15세 관람가,NaN,NaN,[],민감하고 까다로운 여자의 뷰티고민.스토리온이 먼저 체험하고 리뷰한 진짜 뷰티 솔루션...,https://movie-phinf.pstatic.net/20140306_111/1...
2,134204,100인의 선택 - 최고라면,NaN,0.00,15세 관람가,NaN,NaN,"['이창민', '정주리', '파비앙', '김지숙', '록현']",5명의 스타가 일일 셰프가 돼 각 지역의 특산물과 라면을 조합한 '최고의 라면 레시...,https://movie-phinf.pstatic.net/20141202_112/1...
3,123547,100초 전[戰],NaN,0.00,전체 관람가,NaN,NaN,['김진표'],다양한 세대의 실력파 보컬리스트들이 자신을 숨긴 채 출연해 그 가수가 갖고 있는 인...,https://movie-phinf.pstatic.net/20140514_269/1...
4,115240,10분,2014.0,8.83,12세 관람가,['드라마'],이용승,"['백종환', '김종구', '정희태']",방송사 PD시험을 본 ‘호찬(백종환)’은 결과를 기다리며 곧 지방 이전할 공공기관 ...,https://movie-phinf.pstatic.net/20140409_90/13...


In [25]:
df.columns =['movie_id', 'title', 'year','star', 'movie_rating', 'genre', 'director', 'actors', 'summary','image_link']

In [16]:
df

,movie_id,title,year,star,movie_rating,genre,director,actors,summary,image_link,Unnamed: 0,review
0,129765,1000 : 최후의 전사들,2014.0,6.63,15세 관람가,"['액션', '모험', '드라마']",아칸 사타예브,"['아실칸 톨예포프', '아얀 우테부르겐']",칭기즈칸 제국을 재건하려는 준가 족은 카자흐 민족을 학살하고 폭정을 일삼는다. 가...,['https://movie-phinf.pstatic.net/20141014_164...,82.0,최후 전사 카자흐스탄 만들 카자흐스탄 하여 호기심 보게 되었다 카자흐스탄 그리 부유...
1,120699,100인의 뷰티온,NaN,0.00,15세 관람가,NaN,NaN,[],민감하고 까다로운 여자의 뷰티고민.스토리온이 먼저 체험하고 리뷰한 진짜 뷰티 솔루션...,['https://movie-phinf.pstatic.net/20140306_111...,NaN,NaN
2,134204,100인의 선택 - 최고라면,NaN,0.00,15세 관람가,NaN,NaN,"['이창민', '정주리', '파비앙', '김지숙', '록현']",5명의 스타가 일일 셰프가 돼 각 지역의 특산물과 라면을 조합한 '최고의 라면 레시...,['https://movie-phinf.pstatic.net/20141202_112...,NaN,NaN
3,123547,100초 전[戰],NaN,0.00,전체 관람가,NaN,NaN,['김진표'],다양한 세대의 실력파 보컬리스트들이 자신을 숨긴 채 출연해 그 가수가 갖고 있는 인...,['https://movie-phinf.pstatic.net/20140514_269...,NaN,NaN
4,115240,10분,2014.0,8.83,12세 관람가,['드라마'],이용승,"['백종환', '김종구', '정희태']",방송사 PD시험을 본 ‘호찬(백종환)’은 결과를 기다리며 곧 지방 이전할 공공기관 ...,['https://movie-phinf.pstatic.net/20140409_90/...,83.0,안정 나무 찾아 거주 할것인가 성실한 모습 부장 정규직 채용 하려하지만 압력 호찬...
...,...,...,...,...,...,...,...,...,...,...,...,...
2922,111630,히든 스나이퍼,2015.0,6.55,15세 관람가,"['범죄', '드라마', '스릴러']",아네트 K. 올레슨,"['트린 디어홈', '킴 보드니아', '크리스티안 할켄']","당신은 써, 나는 쏠테니. 지구물리학자이자 전 올림픽 사격수인 한 남자. 그린랜드의...",['https://movie-phinf.pstatic.net/20150323_73/...,1406.0,저격자 부천 국제 판타스 영화제 아까 검색 했을 보니 있네 다행 포스팅 시간표 빽빽...
2923,139702,히든싱어 4,NaN,5.00,15세 관람가,NaN,NaN,['전현무'],대한민국을 대표하는 가수와 그 가수의 목소리부터 창법까지 완벽하게 소화 가능한 '모...,['https://movie-phinf.pstatic.net/20150630_249...,NaN,NaN
2924,100647,히말라야,2015.0,7.98,12세 관람가,['드라마'],이석훈,"['황정민', '정우']","해발 8,750 미터 히말라야 에베레스트 데스존인간의 접근을 허락하지 않은 신의 영...",['https://movie-phinf.pstatic.net/20151119_280...,1407.0,히말라야 안녕하세요 월요일 영등포 히말라야 보았습니다 좋아해서 그런지 감동 있게 봤...
2925,137679,히에로니무스 보슈의 쾌락정원,NaN,0.00,15세 관람가,NaN,장 외스타슈,[],"외스타슈가 TV 방송용으로 만든 단편으로, <불쾌한 이야기>에 출연했던 외스타슈의 ...",['https://movie-phinf.pstatic.net/20150320_10/...,NaN,NaN


In [ ]:
df.dropna(inplace=True)

### Movie_ID를 활용해 리뷰 ID(NID) Crawling

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
import pandas as pd
from tqdm import tqdm
# 영화 코드 데이터
df = pd.read_csv('./movie_data_id_2.csv',encoding='utf-8')
print("크롤링 시작..")
c_ = []
cod = []
nim = []
df_2 = {}
for code in df['movie_id']:
    url = f"https://movie.naver.com/movie/bi/mi/review.naver?code={code}"
    gain = requests.get(url)
    html = BeautifulSoup(gain.content, 'html.parser')
    num = html.select_one("#reviewTab > div > div > div.top_behavior > span > em").get_text()
    num = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣]', '', str(num))
    num=int(num)
    if num>100:
        num=100
    #cod.append(code)
    for page in tqdm(range(1, num//10+2)):
        url = f"https://movie.naver.com/movie/bi/mi/review.naver?code={code}&page={page}"
        gain = requests.get(url)
        html = BeautifulSoup(gain.content, 'html.parser')
        html2 = str(html)
        for j in range(10):
            # DDOS 공격 의심을 피하기 위한 sleep
            time.sleep(0.5)
            # 유의미한 내용이 담긴 content까지 접근
            p2 = re.compile(r"showReviewListByNid\('(.*?)'")
            try:
                nim.append(p2.findall(html2)[j])
                url = 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={nim}&code={code}&order=#tab'
                #print(nim)
                #print(p2.findall(html2)[j])
            except:
                continue
        #b_ = [nim]
        #c_.append(b_)  # [[nid]] 형태
        df_2[code] = nim
        nim = []
    print(len(df_2))
print("완료!")


In [ ]:
df_idx = pd.DataFrame([(k,v) for k,v in df_2.items() ])
df_idx['count'] = df_idx[1].apply(lambda x: len(x) )
df_idx = df_idx[df_idx['count'] >= 3]
df_idx.drop('count', axis=1, inplace = True)
df_idx.columns = ['movie_id','nid']

In [ ]:
df = df.merge(df_idx,on='movie_id',how='left')

In [20]:
df.reset_index(inplace=True,drop=True)

In [28]:
df.drop_duplicates(inplace=True)

C:\Users\user\.conda\envs\nlp\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
df.to_csv('./movie_data_crawling.csv',index=False)